In [1]:
from __future__ import print_function
from scipy.io import loadmat
import numpy as np
import tensorflow as tf

/usr/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.logging.set_verbosity(tf.logging.INFO)

In [3]:
SVHN_LOCATION = "/mnt/Data/SVHN/"
BATCH_SIZE = 32
EPOCHS = 20

In [4]:
def preprocessing(img, lbl):
    gray_img = tf.image.rgb_to_grayscale(img)
    norm_img = tf.image.per_image_standardization(gray_img)
    
    tens_location = tf.equal(lbl, 10 * tf.ones(lbl.shape, dtype=tf.uint8))
    tens_with_zeroes = tf.where(tens_location, tf.zeros(lbl.shape, dtype=tf.uint8),
                               lbl)
    
    return gray_img, tens_with_zeroes

In [11]:
def train_input_fn(train_mat, extra_mat):
    train_dataset = tf.data.Dataset.from_tensor_slices((np.moveaxis(train_mat['X'],
        3, 0), train_mat['y'].flatten()))
    extra_dataset = tf.data.Dataset.from_tensor_slices((np.moveaxis(extra_mat['X'],
        3, 0), extra_mat['y'].flatten()))

    #train_dataset = train_dataset.map(
    train_dataset = train_dataset.concatenate(extra_dataset).map(
        preprocessing).shuffle(10000).batch(
        BATCH_SIZE).repeat(EPOCHS - 1)

    return train_dataset

In [6]:
test_input_fn = lambda test_mat: tf.data.Dataset.from_tensor_slices((np.moveaxis(
    test_mat['X'], 3, 0), test_mat['y'].flatten())).map(preprocessing).batch(
    BATCH_SIZE)

In [8]:
def resnet_layer(inputs, filters=16, kernel_size=3, strides=1, activation=tf.nn.relu,
                 batch_normalization=True):
    # Function from keras's example for Cifar10:
    # https://github.com/keras-team/keras/blob/master/examples/cifar10_resnet.py
    #x = tf.layers.conv2d(inputs=inputs, filters=filters, kernel_size=kernel_size,
    #                     strides=strides, padding='same')
    w = tf.Variable(tf.truncated_normal((kernel_size, kernel_size,
                                         int(inputs.shape[-1]), filters)))
    alpha = tf.Variable(tf.truncated_normal((kernel_size, kernel_size,
                                             int(inputs.shape[-1]), filters)))
    hebb = tf.Variable(tf.zeros((kernel_size, kernel_size, int(inputs.shape[-1]),
                                 filters)), trainable=False)
    eta = tf.Variable(tf.truncated_normal([1,]))
    
    b = tf.Variable(tf.truncated_normal((filters,)))
    x = tf.nn.conv2d(input=inputs, filter=w + tf.multiply(alpha, hebb),
                     strides=[1, strides, strides, 1], padding='VALID') + b
    
    y = tf.expand_dims(tf.transpose(x, perm=[0, 3, 1, 2]), axis=-1)
    z = tf.zeros(y.shape)
    expand_output = tf.reshape(tf.concat([y, z], axis=-1), [-1, filters,
        int(x.shape[1]), 2 * int(x.shape[2])])
    
    y = tf.expand_dims(expand_output, axis=-2)
    z = tf.zeros(y.shape)
    expand_output = tf.reshape(tf.concat([y, z], axis=-2), [-1, filters,
        2 * int(x.shape[1]), 2 * int(x.shape[2])])
    expand_output = tf.transpose(expand_output, perm=[0, 2, 3, 1])
    
    multi_output = tf.tile(tf.expand_dims(expand_output, axis=3),
                           [1, 1, 1, int(inputs.shape[-1]), 1])
    hebb = eta * tf.matmul(tf.transpose(inputs), x) + (1 - eta) * hebb #TODO
    
    if batch_normalization:
        x = tf.layers.batch_normalization(x)
    if activation is not None:
        x = activation(x)
    return x

In [14]:
def resnet_model(features, labels, mode):
    # Function from keras's example for Cifar10:
    # https://github.com/keras-team/keras/blob/master/examples/cifar10_resnet.py
    filters = 16
    num_res_blocks = 3
    
    features = tf.cast(features, dtype=tf.float32)
    x = resnet_layer(inputs=features)
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:
                strides = 2
            y = resnet_layer(inputs=x, filters=filters, strides=strides)
            y = resnet_layer(inputs=y, filters=filters, activation=None)
            if stack > 0 and res_block == 0:
                x = resnet_layer(inputs=x, filters=filters, kernel_size=1,
                                 strides=strides, activation=None,
                                 batch_normalization=False)
            x = tf.nn.relu(x + y)
        filters *= 2
    
    x = tf.layers.average_pooling2d(x, pool_size=8, strides=8)
    x = tf.layers.flatten(x)
    
    #x = tf.layers.dense(inputs=x, units=10)
    w = tf.Variable(tf.truncated_normal((int(x.shape[1]), 10)))
    alpha = tf.Variable(tf.truncated_normal((int(x.shape[1]), 10)))
    hebb = tf.Variable(tf.zeros((int(x.shape[1]), 10)), trainable=False)
    eta = tf.Variable(tf.truncated_normal([1,]))
    
    b = tf.Variable(tf.truncated_normal((10,)))
    y = tf.matmul(x, w + tf.multiply(alpha, hebb)) + b
    hebb = eta * tf.reduce_mean(tf.matmul(tf.expand_dims(x, axis=-1),
        tf.expand_dims(y, axis=1)), axis=0) + (1 - eta) * hebb
    x = y
    
    classes = tf.argmax(input=x, axis=1)
    correct_prediction = tf.equal(tf.cast(classes, tf.uint8), labels)
    acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) * 100
    acc = tf.identity(acc, name='accuracy_tensor')
    
    predictions = {'classes': classes, 'accuracy': acc}
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    onehot_labels = tf.one_hot(tf.cast(labels, tf.int32), 10)
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=onehot_labels, logits=x)
    loss = tf.reduce_mean(loss)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.contrib.estimator.TowerOptimizer(tf.train.AdamOptimizer(1e-4))
        train_op = optimizer.minimize(loss=loss,
                                      global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {'accuracy': tf.metrics.accuracy(labels=labels,
                                                       predictions=classes)}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss,
                                      eval_metric_ops=eval_metric_ops)


In [12]:
print("Loading training data...")
train_mat = loadmat(SVHN_LOCATION + "train_32x32.mat")
extra_mat = loadmat(SVHN_LOCATION + "extra_32x32.mat")
#extra_mat = train_mat
print("Loading testing data...")
test_mat = loadmat(SVHN_LOCATION + "test_32x32.mat")
print("All data loaded")

Loading training data...
Loading testing data...
All data loaded


In [13]:
tensors_to_log = {'accuracy': 'accuracy_tensor'}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)

In [15]:
model = tf.estimator.Estimator(model_fn=tf.contrib.estimator.replicate_model_fn(
    resnet_model), model_dir='./')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fad00b1c2b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [16]:
model.train(input_fn=lambda:train_input_fn(train_mat, extra_mat), hooks=[logging_hook])

INFO:tensorflow:Calling model_fn.


ValueError: Dimension 1 in both shapes must be equal, but are 32 and 30. Shapes are [1,32] and [?,30]. for 'MatMul' (op: 'BatchMatMul') with input shapes: [1,32,32,?], [?,30,30,16].

In [ ]:
print(model.evaluate(input_fn=lambda:test_input_fn(test_mat)))